# Relative mutation positions

In [1]:
import pandas as pd
from tqdm.notebook import trange, tqdm

## 1. Prepare CPTAC table

In [3]:
####################
#Load CPTAC dataset
####################

print('Load CPTAC datasets')

CPTAC = ['BRCA', 'CCRCC', 'COAD', 'GBM', 'LUAD', 'OV', 'UCEC']
cptac_df = pd.DataFrame(columns=['gene', 'cpct_aliquot', 'protein_expression', 'aliquot', 'sample',
       'Type', 'median', 'stdev', 'norm_protein_expression', 'log2fpkm',
       'log10fpkm', '#Uploaded_variation', 'Location', 'Allele', 'Feature',
       'protein_mutation', 'Protein_position', 'Amino_acids', 'Consequence',
       'Phenotype', 'Ubiquitinases_Mutated', 'Altered_E3_Ligases',
       'Raw_Residual', 'Stability_Change', 'ABS_Stability_Change'])
for cancer in tqdm(CPTAC, total=len(CPTAC),desc='Load CPTAC data'):
    df = pd.read_csv("/workspace/projects/cptac_analysis/data/"+cancer+"/dataset_irls.gz", sep='\t')
    Dataset = [cancer] * len(df)
    df['Dataset'] = Dataset
    frames = [cptac_df, df]
    cptac_df = pd.concat(frames)
cptac_df.drop_duplicates(keep='first',inplace=True) 

cptac_df['ID'] = cptac_df['gene'].astype(str)+cptac_df['Feature'].astype(str)+cptac_df['sample'].astype(str)+cptac_df['#Uploaded_variation'].astype(str)+cptac_df['Location'].astype(str)

######################################################################################################
#Calculate the mean of rna values for the repeated mutations (log2fpkm, log10fpkm, Stability_Change)
######################################################################################################

print('Eliminate duplicated rna measures')

dupl_samples = ['C3L-00908', 'C3N-00545', 'C3N-01825']

cptac_dupl_df = cptac_df[cptac_df['sample'].isin(dupl_samples)]
df1 = cptac_dupl_df[cptac_dupl_df['Feature'].isnull()].groupby(['gene','sample'],as_index=False).mean()
df2 = cptac_dupl_df[~cptac_dupl_df['Feature'].isnull()].groupby(['gene','Feature','#Uploaded_variation','Location'],as_index=False).mean()
cptac_dupl2_df = pd.concat([df1,df2])
cptac_dupl3_df = cptac_dupl_df.drop(['log2fpkm','log10fpkm','Raw_Residual','Stability_Change','ABS_Stability_Change'],axis=1)
cptac_dupl4_df = pd.merge(cptac_dupl3_df,cptac_dupl2_df,how='left')
cptac_dupl4_df.drop_duplicates(keep='first',inplace=True)
cptac2_df = cptac_df[~(cptac_df['sample'].isin(dupl_samples))]
cptac_df = pd.concat([cptac2_df,cptac_dupl4_df],ignore_index=True)

###########################################################
#Load cdegron table, merge and create cterm_degron column
###########################################################

print('Load cdegron table and merge')

cdegron_df = pd.read_csv(r'/workspace/users/msanchezg/notebooks/cdegron_wtnsfs_annot_cptac.tsv', sep = '\t')
cptac_df = pd.merge(cptac_df,cdegron_df,how='left')

Load CPTAC datasets



Eliminate duplicated rna measures
Load cdegron table and merge


## 2. Prepare CCLE table

In [5]:
###################
#Load CCLE dataset
####################

print('Load CCLE dataset')
ccle_df = pd.read_csv("/workspace/projects/cptac_analysis/data/CCLE/dataset_irls.gz", sep='\t')

ccle_df['ID'] = ccle_df['gene'].astype(str)+ccle_df['Feature'].astype(str)+ccle_df['sample'].astype(str)+ccle_df['#Uploaded_variation'].astype(str)+ccle_df['Location'].astype(str)

#############################################################################################################################
#Calculate the mean of protein values for the repeated mutations (protein_expression, norm_protein_expression, Raw_Residual, Stability_Change, ABS_Stability_Change)
#############################################################################################################################

print('Eliminate duplicated protein measures')

df1 = ccle_df[ccle_df['Feature'].isnull()].groupby(['gene','sample','Phenotype'],as_index=False).mean()
df2 = ccle_df[~ccle_df['Feature'].isnull()].groupby(['gene','Feature','sample','#Uploaded_variation','Location','Phenotype'],as_index=False).mean()
ccle2_df = pd.concat([df1,df2])
ccle3_df = ccle_df.drop(['protein_expression', 'norm_protein_expression', 'Raw_Residual', 'Stability_Change', 'ABS_Stability_Change'],axis=1)
ccle4_df = pd.merge(ccle3_df,ccle2_df,how='left')
ccle_df = ccle4_df.drop_duplicates(keep='first')

################################################################
#Import cterm degrons table, merge and add cterm_degron column
################################################################

print('Load cdegron table and merge')

cdegron_df = pd.read_csv(r'/workspace/users/msanchezg/notebooks/cdegron_wtnsfs_annot_ccle.tsv', sep = '\t')
ccle_df = pd.merge(ccle_df,cdegron_df,how='left')

Load CCLE dataset
Eliminate duplicated protein measures
Load cdegron table and merge


## 3. Get relative mutation position

In [30]:
#Get relative mutation positions for CPTAC dataset (takes a lot of time, 1-5h)
df = cptac_df
phenotypes = ['stop_gained','frameshift_variant','missense_variant','synonymous_variant']
genes_mut_df = df[['gene']][(~df['prot_seq'].isnull())&(df['Phenotype'].isin(phenotypes))]
genes_mut_df = genes_mut_df.drop_duplicates(keep='first')
genes_mut_list = genes_mut_df['gene'].to_list()

#Prepare df all mutations
cptac_prots_mut_df = pd.DataFrame()
for prot in tqdm(genes_mut_list):
    prot_df = df[['ID','Phenotype','prot_seq','Protein_position','Stability_Change','gene']][df['gene']==prot]
    prot_df = prot_df.drop_duplicates(subset=['ID'])
    prot_wt_df = prot_df[prot_df['Phenotype']=='WT']
    prot_wt_list = prot_wt_df.values.tolist()

    prot_mut_df = prot_df[prot_df['Phenotype'].isin(phenotypes)]
    prot_mut_list = prot_mut_df.values.tolist()

    #Distance to cterm from end of sequence

    prot_length = len(prot_wt_list[0][2])

    pos_list = prot_mut_df['Protein_position'].tolist()
    pos1_list = []
    for pos in pos_list:
        pos1 = pos.split('-')[0]
        pos1 = int(pos1)/prot_length
        pos1_list.append(pos1)
    pos1_list
    prot_mut_df['Protein_position2'] = pos1_list

    cptac_prots_mut_df = pd.concat([cptac_prots_mut_df,prot_mut_df], ignore_index=True)

/home/msanchezg/miniconda3/envs/cptac/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [34]:
#Get relative mutation positions for CCLE dataset (takes a lot of time, 1-5h)
df = ccle_df
phenotypes = ['stop_gained','frameshift_variant','missense_variant','synonymous_variant']
genes_mut_df = df[['gene']][(~df['prot_seq'].isnull())&(df['Phenotype'].isin(phenotypes))]
genes_mut_df = genes_mut_df.drop_duplicates(keep='first')
genes_mut_list = genes_mut_df['gene'].to_list()

#Prepare df all mutations
ccle_prots_mut_df = pd.DataFrame()
for prot in tqdm(genes_mut_list):
    prot_df = df[['ID','Phenotype','prot_seq','Protein_position','Stability_Change','gene']][df['gene']==prot]
    prot_df = prot_df.drop_duplicates(subset=['ID'])
    prot_wt_df = prot_df[prot_df['Phenotype']=='WT']
    prot_wt_list = prot_wt_df.values.tolist()

    prot_mut_df = prot_df[prot_df['Phenotype'].isin(phenotypes)]
    prot_mut_list = prot_mut_df.values.tolist()

    #Distance to cterm from end of sequence

    prot_length = len(prot_wt_list[0][2])

    pos_list = prot_mut_df['Protein_position'].tolist()
    pos1_list = []
    for pos in pos_list:
        pos1 = pos.split('-')[0]
        pos1 = int(pos1)/prot_length
        pos1_list.append(pos1)
    pos1_list
    prot_mut_df['Protein_position2'] = pos1_list

    ccle_prots_mut_df = pd.concat([ccle_prots_mut_df,prot_mut_df], ignore_index=True)

/home/msanchezg/miniconda3/envs/cptac/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
cptac_prots_mut_df

,ID,Phenotype,prot_seq,Protein_position,Stability_Change,gene,Protein_position2
0,ABCB7ENST0000064582911BR01611BR016__G__CchrX:7...,stop_gained,MALLAMHSWRWAAAAAAFEKRRHSAILIRPLVSVSGSGPQWRPHQL...,669,-0.501100,ABCB7,0.882586
1,ABCB7ENST0000064582905CO00605CO006__C__TchrX:7...,synonymous_variant,NaN,2,-0.439393,ABCB7,0.002639
2,ABCB7ENST0000064582911CO07011CO070__G__AchrX:7...,missense_variant,NaN,752,-0.636961,ABCB7,0.992084
3,ABCB7ENST0000064582911CO07011CO070__T__GchrX:7...,missense_variant,NaN,110,-0.636961,ABCB7,0.145119
4,ABCB7ENST00000645829C3L-00677C3L-00677__G__Ach...,synonymous_variant,NaN,51,0.797033,ABCB7,0.067282
...,...,...,...,...,...,...,...
81215,UBA2ENST00000246548C3N-00545C3N-00545__-__AAAA...,frameshift_variant,MALSRGLPRELAEAVAGGRVLVVGAGGIGCELLKNLVLTGFSHIDL...,410-411,NaN,UBA2,0.639626
81216,RAB22AENST00000244040C3N-00560C3N-00560__G__Tc...,missense_variant,NaN,107,0.060910,RAB22A,0.548718
81217,RAB22AENST0000024404017OV00217OV002__C__Tchr20...,synonymous_variant,NaN,123,0.190743,RAB22A,0.630769
81218,RAB22AENST00000244040C3L-00586C3L-00586__G__Ac...,missense_variant,NaN,138,0.035960,RAB22A,0.707692


In [6]:
ccle_prots_mut_df

,ID,Phenotype,prot_seq,Protein_position,Stability_Change,gene,Protein_position2
0,SLC12A2ENST00000262461CAL51_BREASTACH-000856__...,synonymous_variant,NaN,477,-0.357224,SLC12A2,0.393240
1,SLC12A2ENST00000262461A2058_SKINACH-000788__C_...,missense_variant,NaN,921,-0.372284,SLC12A2,0.759275
2,SLC12A2ENST00000262461HCT116_LARGE_INTESTINEAC...,missense_variant,NaN,482,-0.818678,SLC12A2,0.397362
3,SLC12A2ENST00000262461RKO_LARGE_INTESTINEACH-0...,missense_variant,NaN,982,-0.584998,SLC12A2,0.809563
4,SLC12A2ENST00000262461HT115_LARGE_INTESTINEACH...,missense_variant,NaN,821,0.068663,SLC12A2,0.676834
...,...,...,...,...,...,...,...
103942,ITGA8ENST00000378076REH_HAEMATOPOIETIC_AND_LYM...,missense_variant,NaN,727,0.285381,ITGA8,0.683271
103943,DLX5ENST00000222598HEC59_ENDOMETRIUMACH-000994...,frameshift_variant,False,143-144,0.195975,DLX5,28.600000
103944,DLX5ENST00000222598HEC59_ENDOMETRIUMACH-000994...,frameshift_variant,False,53,0.195975,DLX5,10.600000
103945,INPP5JENST00000331075HEC59_ENDOMETRIUMACH-0009...,stop_gained,MEGQSSRGSRRPGTRAGLGSLPMPQGVAQTGAPSKVDSSFQLPAKK...,346,-0.261746,INPP5J,0.343595


In [35]:
#Save tables with relative mutation positions
cptac_prots_mut_df.to_csv(r'pos_rel_cptac.tsv', header = True, index = None, sep = '\t')
ccle_prots_mut_df.to_csv(r'pos_rel_ccle.tsv', header = True, index = None, sep = '\t')